# Full respiration model and analytical reductions

This file implements two functions that are required as inputs for ```MBAM```:

* ```BK_simulator(phi::AbstractArray, x::AbstractArray; model_id=0)``` 
For an Mx1 vector of parameters ```phi``` and for a DxN array ```x``` containing (dummy) temperature values and voltages (so D=2) return the Nx1 respiration rate values. Different ```model_id```s describe different parameterisations (reduced models) or assays (actual rate or log rate).

* ```BK_reduce(phi::Vector; model_id=0, in_log=1)``` 
For an Mx1 vector of parameters ```phi``` and for a ```model_id``` return the (M-1)x1 vector of parameters to be used in ```BK_simulator``` for the subsequent model. ```in_log``` describes whether the numerical values of ```phi```s are stored and should be returned after a logarithmic transform (used here for numerical stability due to the diverging parameters).



# MBAM model reduction for the MEND respiration model
Model taken from Wang 2015  PLoS ONE 9(2): e89252.  doi:10.1371/journal.pone.0089252 <br>
Initial parameters taken from Wang 2015, Table 1 <br>
For algorithmic details see the ```MBAM``` notebook.

The goal is to find interpretable models with fewer and fewer parameters in each step.
The iterative model reduction with respect to the heterotrophic respiration rate assay proceeds as follows:
+ model 51 (Original model): <br>
$\theta = \{B_0,\ r_0,\ \mu_G,\ Y_G,\ m_R \}$
    * $B_0 -> 0, \quad Y_G -> 0$
    
    
+ model 10: <br>
$\theta_1 = \{\Phi_1,\ r_0,\ \mu_G,\ m_R \}$
    * $\Phi_1 \rightarrow \infty, \quad \mu_G \rightarrow 0, \quad r_0 \rightarrow 0 $
    

+ model 11: <br>
$\theta_2 = \{\Phi_2,\ \Phi_3,\ \mu_G \}$ 
    * $\Phi_3 \rightarrow 0 $
    
    
+ model 12: <br>
$\theta_3 = \{\Phi_2,\ \mu_G \}$ 
    * $\Phi_2 \rightarrow \infty, \quad \mu_G \rightarrow 0 $



<br>

* Even with 2 parameters ($\theta_2 = \{\Phi_2,\ \mu_G \}$ ) the fit to the data is the same
* The uncertainty in $\mu_G$ is reduced in the reduced model
* $\Phi_2 = B_0r_0/Y_G$ is expressed in terms of the original model parameters
<br>
<br>
+ model 52 corresponds to the log heterotrophic respiration assay

In [1]:
"Grid"
function ndgrid{T}(v1::AbstractArray{T}, v2::AbstractArray{T})
    m, n = length(v1), length(v2)
    v1 = reshape(v1, m, 1)
    v2 = reshape(v2, 1, n)
    (repmat(v1, 1, n), repmat(v2, m, 1))
end

"Return rates for the respiration assay given the (dummy) Temperature and the Time"
function BK_simulator(phi::AbstractArray, x::AbstractArray; model_id=0)
    # Simulates the MWC model (Aldrich 2002) open probabilities of BK given input parameters Phi over the given Ca-V grid points
    # model_id=0 is the full model, further models are reduced representations by MBAM
    
    Temp = vcat(x[1,:]...)
    Time = vcat(x[2,:]...)

    Rate = 0.0;
    
    # ------------------------------------------- # 
    # ----- Base model with 5 parameters -------- #
    # ------------------------------------------- #
    
    if mod(model_id,10)==0
        #phi = {Phi1, r0, muG,mR}
        Phi1 = phi[1]
        r0 = phi[2]
        muG = phi[3]
        mR = phi[4]
        
        Rate = (Phi1).*((muG.*r0 + mR).*exp(muG.*Time) - (mR.*(1-r0)).*exp(-mR.*Time))
        
    end
    
    if model_id==11
        #phi = {Phi2=Phi1*r0, Phi3=Phi1*mR, muG}
        Phi2 = phi[1]
        Phi3 = phi[2]
        muG = phi[3]
        
        Rate = (muG.*Phi2 + Phi3).*exp(muG.*Time) - Phi3
        
    end
    
    if model_id==12
        #phi = {Phi2=Phi1*r0, muG}
        Phi2 = phi[1]
        muG = phi[2]
        
        Rate = (muG.*Phi2).*exp(muG.*Time)
        
    end
    
    
    
    if model_id==51
        #phi = {B0, r0 ,muG, YG, mR}
        B0 = phi[1]
        r0 = phi[2]
        muG = phi[3]
        YG = phi[4]
        mR = phi[5]
        
        Rate = (B0*(1-YG)./YG).*((muG.*r0 + mR).*exp(muG.*Time) - (mR.*(1-r0)).*exp(-mR.*Time));
    end
    
    if model_id==52
        #phi = {B0, r0 ,muG, YG, mR}
        Phi1 = phi[1]
        r0 = phi[2]
        muG = phi[3]
        mR = phi[4]
        
        Rate = (Phi1).*((muG.*r0 + mR).*exp(muG.*Time) - (mR.*(1-r0)).*exp(-mR.*Time));
        Rate = log10(Rate)
    end
        
    
    return Rate
end

    

BK_simulator (generic function with 1 method)

In [ ]:
"Return a reduced parameters vector given a parameter vector that includes diverging dimensions"
function BK_reduce(phi::Vector; model_id=0, in_log=1)
    if in_log==1
        f_add = (a,x,b,y) -> a*x + b*y
    else
        f_add = (a,x,b,y) -> (x.^a)*(y.^b)
    end

    phi_red = zeros(size(phi)[1]-1);
    
    # ------------------------------------------- # 
    # ------- MBAM ran in wrt. open probs ------- #
    # ------------------------------------------- # 
        
    if model_id==10
        #phi = {Phi2=Phi1*r0, Phi3=Phi1*mR, muG}
        phi_red[1] = f_add(1,phi[1],1,phi[2])
        phi_red[2] = f_add(1,phi[1],1,phi[4])
        phi_red[3] = phi[3]
        
    elseif model_id==11
        #phi = {Phi2=Phi1*r0, muG}
        phi_red[1] = phi[1]
        phi_red[2] = phi[3]
    
        
    
        
    elseif model_id==51
        #phi = {Phi1 = B0/YG, r0, muG, mR}
        #phi_red[1] = f_add(1,phi[1],-1,phi[2])
        phi_red[1] = -0.327
        phi_red[2] = log10(0.394)
        phi_red[3] = log10(0.027)
        phi_red[4] = log10(0.00613)
        
        
    else
        
        phi_red = phi;
    end
    
    
   
    
    
    
    return phi_red;
end